# problem1: my rolling first target = last x_array, but in tf target is later
# problem2: my roll deletes values if not enough window, but tf just shrinks window
# roll = convert each row to group of rows
# rolling x: each element gets array of self and history_size-1 previous
# if element has not enough previous - we delete this element

In [56]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import utils_lstm as utils_lstm
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

DATALEN = 100#12#4000#000
#c = self.conf

import numpy as np
import pandas as pd
ser1=pd.Series({'f1':'2018-02-13 08:30:00', 'f2':0, 'f3': 0, 'f4':0.5})
ser2=pd.Series({'f1':'2018-02-13 08:31:00', 'f2':1, 'f3':-1, 'f4':1.5})
ser3=pd.Series({'f1':'2018-02-13 08:32:00', 'f2':2, 'f3':-2, 'f4':2.5})
ser4=pd.Series({'f1':'2018-02-13 08:33:00', 'f2':3, 'f3':-3, 'f4':3.5})
ser5=pd.Series({'f1':'2018-02-13 08:34:00', 'f2':4, 'f3':-4, 'f4':4.5})
ser6=pd.Series({'f1':'2018-02-13 08:35:00', 'f2':5, 'f3':-5, 'f4':5.5})
ser7=pd.Series({'f1':'2018-02-13 08:36:00', 'f2':6, 'f3':-6, 'f4':6.5})
ser8=pd.Series({'f1':'2018-02-13 08:37:00', 'f2':7, 'f3':-7, 'f4':7.5})
ser9=pd.Series({'f1':'2018-02-13 08:38:00', 'f2':8, 'f3':-8, 'f4':8.5})
df = pd.DataFrame([ser1,ser2,ser3,ser4,ser5,ser6,ser7,ser8,ser9])
df['f1'] = pd.to_datetime(df['f1'], errors='coerce')
train_df = df.set_index(df['f1']).drop(columns=['f1'])

def roll_arr_same_size(arr, back_window, fwd_window, step=1, mode='back'):
    data = []
    for i in range(back_window-1,arr.shape[0]-fwd_window+1): # for in range(9): 0 1 ... 8
        if mode=='back':
            good_rowids = [el for el in range(i-back_window+1, i+1, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
            data.append(arr[good_rowids])
        elif mode=='fwd':   
            good_rowids = [el for el in range(i, i+fwd_window, step)] # i=2/h_s=3: (2-3+1,2+1,1); [v for v in range(0, 3, 1)] -> [0, 1, 2]
            data.append(arr[good_rowids]) #train_df.values[[0,1,2,3]] # get first 4 rows
    return np.array(data)    


def prepare_data(train_df): # norm np.arrays
    step = 1
    BUFFER_SIZE = 1000
    BATCH_SIZE = 1
    dataset = train_df.values
    #TRAIN_SPLIT = 4; 
    past_history = 3; 
    future_target = 4
    x_array = dataset[:, 1:]
    y_array = dataset[:, 0]
    
    x_train_multi = roll_arr_same_size(arr=x_array, back_window=past_history, fwd_window=future_target, step=step, mode='back')
    y_train_multi = roll_arr_same_size(arr=y_array, back_window=past_history, fwd_window=future_target, step=step, mode='fwd')
    train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
    train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

    x_val_multi = x_train_multi
    y_val_multi = y_train_multi
    val_data_multi = train_data_multi

    return y_train_multi #x_train_multi.shape,y_train_multi.shape#, y_train_multi  #x_train_multi#,y_val_multi#, y_train_multi

prepare_data(train_df)


array([array([3., 4., 5., 6.]), array([4., 5., 6., 7.]),
       array([5., 6., 7., 8.]), array([6., 7., 8.]), array([7., 8.]),
       array([8.]), array([], dtype=float64)], dtype=object)

In [2]:
range(0,1,6,1)

TypeError: range expected at most 3 arguments, got 4

In [ ]:
[v for v in range(0, 2, 1)]

In [ ]:
#print('x=',prepare_data(train_df)[0],'\n','y=',prepare_data(train_df)[1])
#train_df.index

# history_size = 2
# target_size = 1
# rolling x: each element gets array of self and history_size-1 previous
# if element has not enough previous - we delete this element
#               v    v   v   v   v    v   v    v   - only they have own roll ()
#     f2: |0   -1 | -2  -3  -4  -5 | -6  -7 | -8
#     f3: |0.5 1.5| 2.5 3.5 4.5 5.5| 6.5 7.5| 8.5
#         | rol1  |  ...           |  rol7  | 
        


In [ ]:
# train_data_multi = prepare_data(train_df)
# for x, y in train_data_multi.take(1):
#     print(x.shape)

In [ ]:

df

In [ ]:
df = pd.to_datetime(pd.DataFrame([['2018-02-13 08:30:00'], ['2018-02-13 09:31:00'], ['2018-02-13 10:32:00'], ['2018-02-13 11:33:00'], ['2018-02-13 08:34:00']], columns=['date'])['date'], errors='coerce')
df['val'] = np.random.randn(len(df))
#df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.index = df['date']
df = df.drop(columns=['date'])
df

In [ ]:
dates = pd.to_datetime(pd.date_range('2018-02-13', periods=3, freq='MIN', name='date'), errors='coerce') # freq='T'
col1 = np.random.randn(len(dates))
pd.DataFrame({'activity':col1}, index=dates)

In [ ]:
import numpy as np
import pandas as pd
ser1=pd.Series({'f1':'2018-02-13 08:30:00', 'f2':0, 'f3': 0, 'f4':0.5})
ser2=pd.Series({'f1':'2018-02-13 08:31:00', 'f2':1, 'f3':-1, 'f4':1.5})
ser3=pd.Series({'f1':'2018-02-13 08:32:00', 'f2':2, 'f3':-2, 'f4':2.5})
ser4=pd.Series({'f1':'2018-02-13 08:33:00', 'f2':3, 'f3':-3, 'f4':3.5})
ser5=pd.Series({'f1':'2018-02-13 08:34:00', 'f2':4, 'f3':-4, 'f4':4.5})
ser6=pd.Series({'f1':'2018-02-13 08:35:00', 'f2':5, 'f3':-5, 'f4':5.5})
ser7=pd.Series({'f1':'2018-02-13 08:36:00', 'f2':6, 'f3':-6, 'f4':6.5})
ser8=pd.Series({'f1':'2018-02-13 08:37:00', 'f2':7, 'f3':-7, 'f4':7.5})
ser9=pd.Series({'f1':'2018-02-13 08:38:00', 'f2':8, 'f3':-8, 'f4':8.5})
df = pd.DataFrame([ser1,ser2,ser3,ser4,ser5,ser6,ser7,ser8,ser9])
df['f1'] = pd.to_datetime(df['f1'], errors='coerce')
train_df = df.set_index(df['f1']).drop(columns=['f1'])
train_df = train_df.values

In [ ]:

newrow = [10,20,30]
A = np.vstack([newrow,train_df])
A

In [ ]:
[v for v in range(0, 5, 2)]

In [ ]:
x = np.array([[1,2,2.4],[3,4,5]])
y = np.full((0,x.shape[1]),0)
np.vstack([y,x])

In [ ]:
#     new_rows = np.full((past_history-1,x_array.shape[1]), 0) # generate empty rows at start
#     x_array_prepanded = np.vstack([new_rows,x_array])
    
#     new_targets = np.full((future_target-1,1), 0).reshape(-1) # generate empty rows at start
#     print(new_targets.shape, y_array.shape)
#     y_array_expanded = np.vstack([y_array,new_targets])


In [ ]:
#     x_train_multi, y_train_multi = multivariate_data(
#         x_array,   y_array, 
#         start_index = 0,  end_index = x_array.shape[0]+1,
#         history_size = past_history, target_size = future_target, 
#         step = step,
#     )
    

In [ ]:
# def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
#     data = []; labels = []
#     start_index = start_index + history_size
#     if end_index is None: end_index = len(dataset) - target_size
#     for i in range(start_index, end_index):
#         indices = range(i-history_size, i, step)
#         data.append(dataset[indices])
#         if single_step: labels.append(target[i+target_size])
#         else: labels.append(target[i:i+target_size])
#     return np.array(data), np.array(labels)

